In [1]:
setwd("~/ben_lustre/current_projects/kidney_glomTI_response")

In [2]:
library(ArchR)
set.seed(1)
addArchRThreads(threads = 16) 


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [3]:
addArchRGenome("hg38")

Setting default genome to Hg38.



In [4]:
#get sequencing inventory
seq_inventory = read.csv("~/ben_nfs/sample_donor_data/sequencing_inventory.csv")
seq_inventory = seq_inventory[seq_inventory$experiment %in% "10X_multiome_ATAC", ]
seq_inventory = seq_inventory[!seq_inventory$poor_QC, ]
seq_inventory = seq_inventory[seq_inventory$tissue %in% "kidney_single_nucleus_suspension", ]

In [5]:
input_files = file.path("/lustre/scratch126/cellgen/team297/bs16/current_projects/kidney_glomTI_response/data/seq_data/10X_multiome", seq_inventory$irods_ID, 'atac_fragments.tsv.gz' )
names(input_files) = seq_inventory$sanger_ID

In [6]:
barcode_files = file.path("/lustre/scratch126/cellgen/team297/bs16/current_projects/kidney_glomTI_response/data/seq_data/10X_multiome", seq_inventory$irods_ID, 
                         'filtered_feature_bc_matrix', 'barcodes.tsv.gz')
names(input_files) = seq_inventory$sanger_ID

In [7]:
valid_barcodes = lapply(barcode_files, function(x){
    read.csv(x, header = FALSE, sep = '\t')[, 1]
})

In [8]:
#we do this in a dedicated dir because it creates all sorts of messy files.
setwd("/lustre/scratch126/cellgen/team297/bs16/current_projects/kidney_glomTI_response/data/raw/10X_multiome/archR_files")
#make the arrow files 
ArrowFiles <- createArrowFiles(
  inputFiles = input_files,
  sampleNames = names(input_files),
    validBarcodes = valid_barcodes, 
    minTSS = 0,
  minFrags = 0, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-332b4fd54acf-Date-2023-02-08_Time-16-43-03.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2023-02-08 16:43:03 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-332b4fd54acf-Date-2023-02-08_Time-16-43-03.log



In [9]:
ArrowFiles

[1] "Well_KID11925175.arrow" "Well_KID11925176.arrow" "Well_KID11925177.arrow"
 [4] "Well_KID11925174.arrow" "Well_KID10340601.arrow" "Well_KID10340604.arrow"
 [7] "Well_KID10340606.arrow" "Well_KID10340597.arrow" "Well_KID10340603.arrow"
[10] "Well_KID10340605.arrow" "Well_KID10340598.arrow" "Well_KID10340602.arrow"
[13] "Well_KID10340595.arrow" "Well_KID10340596.arrow" "Well_KID10340599.arrow"
[16] "Well_KID10340600.arrow"

In [10]:
#this is a vector of file paths
ArrowFiles
saveRDS(ArrowFiles, "./arrow_files.RDS")

[1] "Well_KID11925175.arrow" "Well_KID11925176.arrow" "Well_KID11925177.arrow"
 [4] "Well_KID11925174.arrow" "Well_KID10340601.arrow" "Well_KID10340604.arrow"
 [7] "Well_KID10340606.arrow" "Well_KID10340597.arrow" "Well_KID10340603.arrow"
[10] "Well_KID10340605.arrow" "Well_KID10340598.arrow" "Well_KID10340602.arrow"
[13] "Well_KID10340595.arrow" "Well_KID10340596.arrow" "Well_KID10340599.arrow"
[16] "Well_KID10340600.arrow"

In [11]:
#make a file of arrow paths
base = '/lustre/scratch126/cellgen/team297/bs16/current_projects/kidney_glomTI_response/data/raw/10X_multiome/archR_files'
fp = file.path(base, ArrowFiles)
#save this out
write.table(fp, file.path(base, "arrow_paths.txt"),
          sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE, quote = FALSE)
